In [5]:
import torch

In [5]:
import os

# 添加环境变量
new_path = r'E:\GitHub\image-deduplicate-cluster-webui\venv\Lib\site-packages\torch\lib'
os.environ['PATH'] += os.pathsep + new_path

# 打印环境变量
print(os.environ['PATH'])

e:\GitHub\image-deduplicate-cluster-webui\venv\Scripts;E:\GitHub\image-deduplicate-cluster-webui\venv\Scripts;E:\VMware\VMware Player\bin\;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Windows\System32\OpenSSH\;C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\WINDOWS\System32\OpenSSH\;E:\Program Files\MATLAB\R2022a\runtime\win64;E:\Program Files\MATLAB\R2022a\bin;C:\Program Files (x86)\NoteBook FanControl\;C:\Program Files\dotnet\;D:\Git\cmd;C:\MinGW\bin;C:\Users\WSH\AppData\Local\Programs\Python\Python310\Scripts\;C:\Users\WSH\AppData\Local\Programs\Python\Python310\;D:\Cadence_SPB_16.6\Cadence\Cadence_SPB_16.6\openaccess\bin\win32\opt;D:\Cadence_SPB_16.6\Cadence\Cadence_SPB_16.6\tools\capture;D:\Cadence_SPB_16.6\Cadence\Cadence_SPB_16.6\tools\pspice;D:\Cadence_SPB_16.6\C

In [6]:
import tensorrt as trt

print("TensorRT version: {}".format(trt.__version__))

TensorRT version: 8.6.1


In [6]:
import onnxruntime as ort

# 打印 ONNX Runtime 版本信息
print("ONNX Runtime version:", ort.__version__)

# 检查是否可用 GPU
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is NOT available")

import os
model_path = r"E:\GitHub\image-deduplicate-cluster-webui\wd14_tagger_model\model.onnx"

ONNX Runtime version: 1.15.0
GPU is available


In [7]:
# 缓存在与onnx模型同目录下的'trt_engine_cache'文件夹中
trt_engine_cache_path = os.path.join( os.path.dirname(model_path), "trt_engine_cache" )
Tensorrt_options = {"trt_timing_cache_enable": True,  # 时序缓存,可以适用于多个模型
                    "trt_engine_cache_enable": True,  # 开启引擎缓存,针对特定模型、推理参数、GPU
                    "trt_engine_cache_path":trt_engine_cache_path,
                    "trt_fp16_enable": False,  # FP16模式，需要GPU支持
                    "trt_int8_enable": False,  # INT8模式，需要GPU支持
                    "trt_dla_enable": False,  # DLA深度学习加速器，需要GPU支持
                    "trt_build_heuristics_enable" : False,  # 启用启发式构建，减少时间
                    "trt_builder_optimization_level": 3,  # 优化等级，越小耗时越少，但优化更差，不建议低于3
}
Tensorrt_provider = ("TensorrtExecutionProvider", Tensorrt_options)

if Tensorrt_options["trt_engine_cache_enable"]:
    print(f"""
Your cache files will be stored in {Tensorrt_options["trt_engine_cache_path"]}
Warning: Please clean up any old engine and profile cache files (.engine and .profile) if any of the following changes:
1.Model changes (if there are any changes to the model topology, opset version, operators etc.)
2.ORT version changes (i.e. moving from ORT version 1.8 to 1.9)
3.TensorRT version changes (i.e. moving from TensorRT 7.0 to 8.0)
4.Hardware changes. (Engine and profile files are not portable and optimized for specific Nvidia hardware)
""")


Your cache files will be stored in E:\GitHub\image-deduplicate-cluster-webui\wd14_tagger_model\trt_engine_cache
1.Model changes (if there are any changes to the model topology, opset version, operators etc.)
2.ORT version changes (i.e. moving from ORT version 1.8 to 1.9)
3.TensorRT version changes (i.e. moving from TensorRT 7.0 to 8.0)
4.Hardware changes. (Engine and profile files are not portable and optimized for specific Nvidia hardware)



In [8]:
print("注册的执行者", ort.get_available_providers())

use_cuda = True
use_tensorrt = True

EP_list = ['CPUExecutionProvider']
if use_cuda:
    EP_list += ['CUDAExecutionProvider']
if use_tensorrt:
    EP_list += [Tensorrt_provider]

print("最终选择的执行者", EP_list)

# initialize the model.onnx
sess = ort.InferenceSession(model_path, providers=[Tensorrt_provider])


可用执行者 ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [9]:
import numpy as np
import time
# 获取输出
start = time.time()

# get the outputs metadata as a list of :class:`onnxruntime.NodeArg`
output_name = sess.get_outputs()[0].name
for i in range(360):
    input_arr = np.random.randint(0, 225, size=(1, 448, 448, 3)).astype(np.float32)
    ort_inputs = {sess.get_inputs()[0].name: input_arr,}
    ort_outs = sess.run(None, ort_inputs)
print("cost time:", time.time() - start)
print(ort_outs)
print(ort_outs[0].shape)

cost time: 21.170454740524292
[array([[7.8934796e-02, 5.6522924e-01, 2.3527807e-01, ..., 6.0830089e-06,
        3.2220836e-05, 4.7021302e-05]], dtype=float32)]
(1, 9083)
